In [8]:
import pandas as pd 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [9]:
df = pd.read_csv("Dataset/spotify_dataset.csv")

# Tranformation des Dataset

### Spotify_dataset_transformed

In [12]:
top_genres = ['hip hop', 'pop', 'rock', 'trap', 'alternative rock', 'rap', 'pop rock', 'electronic']
df_copy = df.copy()
def get_most_popular_genre(genres_str):
    if pd.isna(genres_str):
        return None
    genres = [g.strip() for g in genres_str.split(',')]
    for top_genre in top_genres:
        if top_genre in genres:
            return top_genre
    return None  # Aucun des top genres n'est présent

# Appliquer la fonction à la colonne contenant les genres
df_copy['Main_genre'] = df_copy['Genre'].apply(get_most_popular_genre)

# Afficher les premières lignes pour vérifier

df_copy[['Genre', 'Main_genre']]
nb_none = df_copy['Main_genre'].isna().sum()
df_transformed = df.dropna(subset=['Main_genre']).copy()
print(f"Nombre de lignes avant suppression : {nb_lines}")
print(f"Nombre de lignes après suppression : {len(df_transformed)}")


main_genre_mapping = {genre: idx for idx, genre in enumerate(top_genres)}
df_transformed["Main_genre_index"] = df_transformed["Main_genre"].map(main_genre_mapping)

df_transformed[list(set((desc_features + ['Main_genre',"Main_genre_index"]))-set(["Album","Popularity","Release Date"]))]

KeyError: ['Main_genre']

### Spotify_dataset_popularity_balanced

In [ ]:
import pandas as pd

# Chargement du dataset
df = pd.read_csv("Dataset/spotify_dataset.csv")  # Remplace par le bon chemin si besoin


def length_to_seconds(length):
    if pd.notna(length):
        parts = str(length).split(":")
        if len(parts) == 2:
            minutes = int(parts[0])
            seconds = int(parts[1])
            return minutes * 60 + seconds
    return 0

df['text'] = df['text'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0) #A Réécrire
df['Length'] = df['Length'].apply(length_to_seconds) #Conversion du Length en secondes

# Séparer les morceaux populaires et non populaires
popular = df[df["Popularity"] > 70]
non_popular = df[df["Popularity"] <= 70]

# Objectif : 40% de morceaux populaires, 60% de non populaires
nb_popular = 8000
nb_non_popular = int((0.6 * nb_popular) / 0.4)  # soit 15000

# Échantillonnage aléatoire
popular_sampled = popular.sample(n=nb_popular, random_state=42)
non_popular_sampled = non_popular.sample(n=nb_non_popular, random_state=42)

# Fusionner et mélanger
balanced_df = pd.concat([popular_sampled, non_popular_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# Vérification
l1 = len(balanced_df[balanced_df["Popularity"] > 70])
l2 = len(balanced_df[balanced_df["Popularity"] <= 70])
print("Total :", len(balanced_df))
print("Popularité > 70 :", l1)
print("Popularité ≤ 70 :", l2)
print("Proportions :", round(l1 / (l1 + l2), 2), "/", round(l2 / (l1 + l2), 2))

df.to_csv("Dataset/spotify_dataset_popularity_balanced.csv", index=False)